# Introduction
Nowadays, the world revolves around data. We created elaborate infrastructures, namely databases, to interact with the data in efficient ways. Yet,this requires more sophisticates techniques like mastering SQL access the data. THis locks away the information for people that lack knowledge of SQL. It would be useful to have a tool that enables everyone to access and interact with the databases. This short project looks into Large Language Models (LLMs) and how they can help use to interact with databases. 
This project has two parts. First, we'll let a LLM interact with a dataset, which generates features like a sentiment of a dialogue and names involved persons. Is uses the LLM capabilty of function calling to built a dialgue processing system. The second part explores the ability of an LLM to interact with a database. 
I used  Nexus Raven for this project. It has 13B parameter, yet it is specifically trained on python code to enhance the performance in calling functions. If you want to find out more, check out their [course](https://www.deeplearning.ai/short-courses/function-calling-and-data-extraction-with-llms/) on Deeplearning.ai.





# Dataset
The Customer Service dataset is freely available from [HuggingFace](https://huggingface.co/datasets/SantiagoPG/customer_service_chatbot). It contains 1000 records of interactions of custumers and customer service containing data about the type of issue, product category and more. I will be focussing on the conversation dialogue that happened between both parties. This allows for interesting applications of LLMs to extract features from text.

The other database I used in the project is the Chinook database. It contains several tables which allows to build more ellaborate SQL queries including joins.

# Using LLM to interact with the data to create a new dataset

Let's have a quick look at the dataset and see how we can you an LLM to call functions to interact with the data.

First, we print out the a sample and look at the conversation, that we'll be using to create a sentiment prediction.



In [52]:
from datasets import load_dataset
import os

cwd = os.getcwd()
# load the customer service chatbot dataset
dialogue_data = load_dataset(cwd + "/data/customer_service_chatbot", cache_dir="./cache")["train"]
sample = dialogue_data[6] # load a sample 
dialogue_string = sample["conversation"].replace("\n\n", "\n") # remove double newlines
print (dialogue_string)


Customer: Hi, I am facing an issue while logging into my account. I am getting an error message saying that I have exceeded the number of attempts to enter the correct verification code.
Agent: Hi there, I'm sorry to hear that you are facing issues with logging into your account. I'll be happy to assist you with that. Can you please provide me with your email address and the order number for the water purifier?
Customer: Sure, my email address is john.doe@gmail.com, and my order number is BB98765432.
Agent: Thank you for providing that information. Let me check your account and see what I can do for you.
(Agent puts the customer on hold for a minute)
Agent: Thank you for holding. I have checked your account, and I can see that there were multiple attempts to enter the verification code. I suggest you wait for a few minutes and try again. If you are still unable to log in, I can reset your password and send you a new verification code.
Customer: Okay, I will try again in a few minutes.


True

For the LLM to extract features from these dialogues, we create a data class "Record" which describes the desired format and features that should be extracted.

In [3]:
from dataclasses import dataclass, fields
dataclass_schema_representation = '''
@dataclass
class Record:
    agent_name : str # The agent name
    customer_email : str # customer email if provided, else ''
    customer_order : str # The customer order number if provided, else ''
    customer_phone : str # the customer phone number if provided, else ''
    customer_sentiment : str # Overall customer sentiment, either 'frustrated', or 'happy'. Always MUST have a value.
'''

# Let's call exec to insert the dataclass into our python interpreter so it understands this. 
exec(dataclass_schema_representation)

This makes it easy for the LLM to add records to a database. We will now initialize a database where the extracted features will be stored. 

In [4]:
def initialize_db():
    import sqlite3

    # Connect to SQLite database (or create it if it doesn't exist)
    conn = sqlite3.connect('extracted_test.db')
    cursor = conn.cursor()

    # Fixed table name
    table_name = "customer_information"

    # Fixed schema
    columns = """
    id INTEGER PRIMARY KEY, 
    agent_name TEXT, 
    customer_email TEXT, 
    customer_order TEXT, 
    customer_phone TEXT, 
    customer_sentiment TEXT
    """

    # Ensure the table name is enclosed in quotes if it contains special characters
    quoted_table_name = f'"{table_name}"'

    # Check if a table with the exact name already exists
    cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name={quoted_table_name}")
    if cursor.fetchone():
        print(f"Table {table_name} already exists.")
    else:
        # Create the new table with the fixed schema
        cursor.execute(f'''CREATE TABLE {quoted_table_name} ({columns})''')
        print(f"Table {table_name} created successfully.")

    # Commit the transaction and close the connection
    conn.commit()
    conn.close()

In [5]:
!rm extracted_test.db
initialize_db()

Table customer_information created successfully.


We also need a function which allows us to populate the database.

In [6]:

from typing import List
def update_knowledge(results_list : List[Record]):
    """
    Registers the information necessary
    """
    import sqlite3
    from sqlite3 import ProgrammingError

    # Reconnect to the existing SQLite database
    conn = sqlite3.connect('extracted_test.db')
    cursor = conn.cursor()

    # Fixed table name
    table_name = "customer_information"

    # Prepare SQL for inserting data with fixed column names
    column_names = "agent_name, customer_email, customer_order, customer_phone, customer_sentiment"
    placeholders = ", ".join(["?"] * 5) 
    sql = f"INSERT INTO {table_name} ({column_names}) VALUES ({placeholders})"

    # Insert each record
    for record in results_list:
        try:
            record_values = tuple(getattr(record, f.name) for f in fields(record))
            cursor.execute(sql, record_values)
        except ProgrammingError as e:
            print(f"Error with record. {e}")
            continue

    # Commit the changes and close the connection
    conn.commit()
    conn.close()
    print("Records inserted successfully.")

Now this is where it becomes interesting. We'll define a prompt which enables the LLM to extract the information. For this we take the signature and the docstring of the update_knowledge function as well as the data class "Record" to provide more information for the LLM on how to call the function update_knowledge.


In [8]:
import inspect
prompt = "\n" + dialogue_string

signature = inspect.signature(update_knowledge)
signature = str(signature).replace("__main__.Record", "Record")
docstring = update_knowledge.__doc__

raven_prompt = f'''{dataclass_schema_representation}\nFunction:\n{update_knowledge.__name__}{signature}\n    """{docstring}"""\n\n\nUser Query:{prompt}<human_end>'''
print (raven_prompt)


@dataclass
class Record:
    agent_name : str # The agent name
    customer_email : str # customer email if provided, else ''
    customer_order : str # The customer order number if provided, else ''
    customer_phone : str # the customer phone number if provided, else ''
    customer_sentiment : str # Overall customer sentiment, either 'frustrated', or 'happy'. Always MUST have a value.

Function:
update_knowledge(results_list: List[Record])
    """
    Registers the information necessary
    """


User Query:
Agent: Hello, thank you for contacting BrownBox customer support. My name is Alex, how can I assist you today?
Customer: Hi, I'm calling about my order for a water purifier. I received it yesterday, but it's not working correctly. I want to return it and get a refund.
Agent: I'm sorry to hear that. I'll be happy to help you with that. Can you please provide me with your order number?
Customer: Sure, it's 12345.
Agent: Thank you for the information. May I know the reason for th

We can now run the query using the LLM. The LLM correctly populates the record and passes it to the update_knowledge function. We use the eval() function to run the function and to add the record to the database.

In [9]:
from utils import query_raven
raven_call = query_raven(raven_prompt)
print (raven_call)

update_knowledge(results_list=[Record(agent_name='Alex', customer_email='', customer_order='12345', customer_phone='', customer_sentiment='frustrated')])


In [14]:
eval(raven_call)

Records inserted successfully.


To check the new entry, we need a function to interact with the database:

In [10]:
import sqlite3
def execute_sql(sql: str):
    """ Runs SQL code for the given schema. Make sure to properly leverage the schema to answer the user's question in the best way possible. """
    # Fixed table name, assuming it's not dynamically generated anymore
    table_name = "customer_information"

    # Establish a connection to the database
    conn = sqlite3.connect('extracted_test.db')
    cursor = conn.cursor()

    # Execute the SQL statement
    cursor.execute(sql)

    # Initialize an empty list to hold query results
    results = []

    results = cursor.fetchall()
    print("Query operation executed successfully. Number of rows returned:", len(results))

    # Close the connection to the database
    conn.close()

    # Return the results for SELECT operations; otherwise, return an empty list
    return results


In [15]:
sql = '''
    SELECT agent_name 
        FROM customer_information
        WHERE customer_sentiment = "frustrated"
    '''
# Print the final SQL command for debugging
print("Executing SQL:", sql)

execute_sql(sql)

Executing SQL: 
    SELECT agent_name 
        FROM customer_information
        WHERE customer_sentiment = "frustrated"
    
Query operation executed successfully. Number of rows returned: 1


[('Alex',)]

I have created a python file which allows the user to create a database called extracted.db. It can be called in the terminal using

```
python3 createDatabase.py --n=10
``` 
with n being the number of records created from the original dataset, which contains 1000 records.

**Note:** Everything so far was done using a test database called extracted_test.db. From now on, we'll use the database that is created using the script (specifically with n=10).

# Exploring the database


Let's check out the newly created database extracted.db. We'll do so using the function execute_sql, get_query that is defined in the utils file. These functions are more versatile letting the user choose the database.



In [3]:
from utils import execute_sql, get_query

## Database created from LLM

In the cell below are three questions. The first one creates an overview of the database. The LLM performed well in extracting the data from the dialogue information. For entry 4, it does not identify the name correctly ("BrownBox" although the agent name was never mentioned). This could potentially be fixed by providing a more specific prompt in that regard. 
The second question one probes the LLM with a more challenging question. The LLM correctly creates a WHERE statement. It manages to understand what a gmail email adress is, i.e. a adress that ends in gmail.com which is checked using the LIKE statement.
The third question does not returns an empty list, although there is a phone number that starts with +1

SQL QUERY RETURN uses '' instead of "" which makes it impossible to run a query like 'SELECT * FROM customer_information WHERE customer_phone LIKE '+1%'' because of %

In [4]:
question = "Give me all the entries from the database?"
question = "What are the names of customers with a gmail email adress?"
#question = "Give me all the entries from the database with an American phone number, which starts with '+1'?"
raven_call, prompt = get_query(database_path = "data/extracted.db", question = question)
print(raven_call)
eval(raven_call)


execute_sql(sql='SELECT customer_email FROM customer_information WHERE customer_email LIKE "%gmail.com"', database_path='data/extracted.db')
Query operation executed successfully. Number of rows returned: 1


[('john.doe@gmail.com',)]

In [57]:
execute_sql(sql="SELECT * FROM customer_information WHERE customer_phone LIKE '+1%'", database_path='data/extracted.db')


Query operation executed successfully. Number of rows returned: 1


[(1, 'Tom', 'johndoe@email.com', '', '+1 123-456-7890', 'happy')]

## Chinook database
The chinook database is a classic example database which represents a digital media store. It contains mutliple tables.
![Chinook](img/chinook_db.png)
This allows us to really see the abilities of LLMs from complex tasks. Let's create three queries of increasing complexity which we can later compare to the LLM.

In [5]:
question = "Give me all the entries from the artist table? Sort the names alphabetically."
question = "What the tables in the database?"
#question = "Give me all the entries from the database with an American phone number, which starts with '+1'?"
raven_call, prompt = get_query(database_path = "data/Chinook.db", question = question)
print(raven_call)
eval(raven_call)

execute_sql(sql='SELECT name FROM sqlite_master WHERE type = "table"', database_path='data/Chinook.db')
Query operation executed successfully. Number of rows returned: 11


[('Album',),
 ('Artist',),
 ('Customer',),
 ('Employee',),
 ('Genre',),
 ('Invoice',),
 ('InvoiceLine',),
 ('MediaType',),
 ('Playlist',),
 ('PlaylistTrack',),
 ('Track',)]

# Pushing the LLM

We have now seen that the LLM is capable of performing simple tasks to extract data from a database. Let's see what it can do to for more complicated questions. I'll be using the Chinook database as it contains multiple tables.

## Simple Joins






In [6]:
question = "What is the genre of the track with the longest duration?" 
question = "List the tracks and their genre? List 10 entries"
question = "Count the number of artists for the genre of each track?" 

question = "What are the genre names in the database?"
question = "COunt the number of tracks for each genre name?" #works
#question = "Give me all the entries from the database with an American phone number, which starts with '+1'?"
raven_call, prompt = get_query(database_path = "data/Chinook.db", question = question)
print(raven_call)
eval(raven_call)

execute_sql(sql='SELECT Genre.Name, COUNT(Track.Name) FROM Genre JOIN Track ON Genre.GenreId = Track.GenreId GROUP BY Genre.Name', database_path='data/Chinook.db')
Query operation executed successfully. Number of rows returned: 25


[('Alternative', 40),
 ('Alternative & Punk', 332),
 ('Blues', 81),
 ('Bossa Nova', 15),
 ('Classical', 74),
 ('Comedy', 17),
 ('Drama', 64),
 ('Easy Listening', 24),
 ('Electronica/Dance', 30),
 ('Heavy Metal', 28),
 ('Hip Hop/Rap', 35),
 ('Jazz', 130),
 ('Latin', 579),
 ('Metal', 374),
 ('Opera', 1),
 ('Pop', 48),
 ('R&B/Soul', 61),
 ('Reggae', 58),
 ('Rock', 1297),
 ('Rock And Roll', 12),
 ('Sci Fi & Fantasy', 26),
 ('Science Fiction', 13),
 ('Soundtrack', 43),
 ('TV Shows', 93),
 ('World', 28)]

## Multiple joins
The performance seems to rapidly drop for more complex queries involving multiple joins. A common mistake is that the LLM starts to use column names of the wrong table, i.e. Album.GenreId. Or it simply ignores the part of the query.

In [7]:
question = "Get the artist name and genre of each track?" #does not work: requires a join of Artist, Album, Track and Genre tables
question = "Get the album name of each track and its genre name? This requires a join of the Album and the Genre table with the Track table" #does not work: requires a join of Album, Track and Genre tables
question = "What is the duration of the tracks from the rock genre? List only the tracks that are longer than 5 minutes."  #works
question = "What is the duration of the tracks from the rock genre that are in playlists with an Id smaller that 5?" #lets check this complex query



raven_call, prompt = get_query(database_path = "data/Chinook.db", question = question)
print(raven_call)
eval(raven_call)


execute_sql(sql='SELECT Track.Name, Track.Milliseconds FROM Track INNER JOIN PlaylistTrack ON Track.TrackId = PlaylistTrack.TrackId INNER JOIN Playlist ON PlaylistTrack.PlaylistId = Playlist.PlaylistId WHERE Playlist.PlaylistId < 5 AND Track.GenreId = 1', database_path='data/Chinook.db')
Query operation executed successfully. Number of rows returned: 1297


[('For Those About To Rock (We Salute You)', 343719),
 ('Balls to the Wall', 342562),
 ('Fast As a Shark', 230619),
 ('Restless and Wild', 252051),
 ('Princess of the Dawn', 375418),
 ('Put The Finger On You', 205662),
 ("Let's Get It Up", 233926),
 ('Inject The Venom', 210834),
 ('Snowballed', 203102),
 ('Evil Walks', 263497),
 ('C.O.D.', 199836),
 ('Breaking The Rules', 263288),
 ('Night Of The Long Knives', 205688),
 ('Spellbound', 270863),
 ('Go Down', 331180),
 ('Dog Eat Dog', 215196),
 ('Let There Be Rock', 366654),
 ('Bad Boy Boogie', 267728),
 ('Problem Child', 325041),
 ('Overdose', 369319),
 ("Hell Ain't A Bad Place To Be", 254380),
 ('Whole Lotta Rosie', 323761),
 ('Walk On Water', 295680),
 ('Love In An Elevator', 321828),
 ('Rag Doll', 264698),
 ('What It Takes', 310622),
 ('Dude (Looks Like A Lady)', 264855),
 ("Janie's Got A Gun", 330736),
 ("Cryin'", 309263),
 ('Amazing', 356519),
 ('Blind Man', 240718),
 ('Deuces Are Wild', 215875),
 ('The Other Side', 244375),
 ('Craz

# Summary

The final performance of the LLM is very robust for simpler tasks. I am sure that more elaborate prompts will help the LLM to get more accurate results on more complicated questions.
The created functions come in very handy in first interacting with a new database performing easier queries. It only needs the following codeblock which can be copied in a Jupyter notebook to first get a sense of a new data base. 


In [8]:
from utils import execute_sql, get_query

question = "What is the name and email of each employee?"
raven_call, prompt = get_query("./data/Chinookyyy.db", question)   
print(raven_call)
print(eval(raven_call))

execute_sql(sql='SELECT LastName, FirstName, Email FROM Employee', database_path='./data/Chinookyyy.db')
Query operation executed successfully. Number of rows returned: 8
[('Adams', 'Andrew', 'andrew@chinookcorp.com'), ('Edwards', 'Nancy', 'nancy@chinookcorp.com'), ('Peacock', 'Jane', 'jane@chinookcorp.com'), ('Park', 'Margaret', 'margaret@chinookcorp.com'), ('Johnson', 'Steve', 'steve@chinookcorp.com'), ('Mitchell', 'Michael', 'michael@chinookcorp.com'), ('King', 'Robert', 'robert@chinookcorp.com'), ('Callahan', 'Laura', 'laura@chinookcorp.com')]




I would like to extend this project to create a LLM Agent that uses the Llama3.1 model to compared the performance. This could be a set up for a Gradio app that let's you upload .csv or .xlsx files which then creates a database. It would also be interesting to get more intuition behind the extracted information. An LLM agent could summarize and explain the data it sees in more detail.